# Prework

## Functions and libraries

In [1]:
import os
# PATH = "/Users/luanagiusto/TP-1-ML"  # Cambia esto si tu path es diferente
PATH = "C:/Users/julia/OneDrive/Escritorio/Archivos/Capacitación/Maestría/03. Machine Learning/TP"

In [ ]:
import pandas as pd
import numpy as np
# from ydata_profiling import ProfileReport
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
pd.set_option('display.width', 1000)
pd.set_option('display.expand_frame_repr', False)
import gc
# Nota: Antes de ejecutar este notebook, instala los requisitos con:
# !pip install -r requirements.txt

In [3]:
def data_profiling(df, output_file):
    # Opciones para que sea liviano
    profile = ProfileReport(
        df.sample(20000, random_state=42) if len(df) > 20000 else df,
        title=output_file,
        minimal=True,         # desactiva análisis costosos
        explorative=True      # agrega secciones útiles
    )

    profile.to_file(output_file)  # <-- abre este HTML en el navegador

In [4]:
# Funcion para mostrar un resumen del dataframe
def df_info_summary(df: pd.DataFrame):
    total = len(df)
    non_null = df.notnull().sum()
    nulls = df.isnull().sum()
    dtypes = df.dtypes
    
    resumen = pd.DataFrame({
        "Non-Null Count": non_null,
        "Null Count": nulls,
        "% Null": (nulls / total * 100).round(2),
        "Dtype": dtypes
    })
    print(resumen)

In [5]:
def resumir_por_id(df, id_col='ID', excluir_cols=None, verbose=False, nombre_conteo='n_registros'):
    """
    Sumariza un DataFrame agrupando por una columna ID.
    Calcula métricas estadísticas básicas para columnas numéricas,
    excluyendo las que se indiquen. Incluye conteo total de registros por ID.

    Parámetros:
    - df: DataFrame de entrada con múltiples registros por ID.
    - id_col: nombre de la columna que identifica cada entidad única.
    - excluir_cols: lista de columnas a excluir del resumen (opcional).
    - verbose: si True, imprime columnas incluidas y excluidas.
    - nombre_conteo: nombre de la columna que indica cantidad de registros por ID.

    Retorna:
    - DataFrame con una fila por ID y métricas estadísticas por columna.
    """
    if excluir_cols is None:
        excluir_cols = []

    excluir_set = set(excluir_cols)
    if id_col in excluir_set:
        excluir_set.remove(id_col)

    numeric_cols = df.select_dtypes(include=np.number).columns.tolist()
    cols_a_resumir = [col for col in numeric_cols if col not in excluir_set and col != id_col]

    if verbose:
        print(f"Columnas excluidas: {sorted(excluir_set)}")
        print(f"Columnas resumidas: {sorted(cols_a_resumir)}")

    # Agregaciones estadísticas
    agg_funcs = ['mean', 'min', 'max', 'median', 'sum']
    agg_dict = {col: agg_funcs for col in cols_a_resumir}

    # Agregar conteo de registros por ID
    df[nombre_conteo] = 1
    agg_dict[nombre_conteo] = ['count']

    resumen = df.groupby(id_col).agg(agg_dict)
    resumen.columns = [f"{col}_{stat}" for col, stat in resumen.columns]
    resumen = resumen.reset_index()

    return resumen

## Data import and overview

In [6]:
# PATH = "/Users/luanagiusto/TP-1-ML"  # Cambia esto si tu path es diferente
PATH = "C:/Users/julia/OneDrive/Escritorio/Archivos/Capacitación/Maestría/03. Machine Learning/TP"
train_df = pd.read_parquet(os.path.join(PATH, "train.parquet"), engine='fastparquet')

In [7]:
df_info_summary(train_df)

                                                    Non-Null Count  Null Count  % Null    Dtype
SK_ID_CURR                                                  307511           0    0.00    int64
TARGET                                                      307511           0    0.00    int64
NAME_CONTRACT_TYPE                                          307511           0    0.00   object
CODE_GENDER                                                 307511           0    0.00   object
FLAG_OWN_CAR                                                307511           0    0.00   object
FLAG_OWN_REALTY                                             307511           0    0.00   object
CNT_CHILDREN                                                307511           0    0.00    int64
AMT_INCOME_TOTAL                                            307511           0    0.00  float64
AMT_CREDIT                                                  307511           0    0.00  float64
AMT_ANNUITY                             

In [8]:
# Por ahora reemplazo nan con ceros, pero habría que ver si se puede mejorar
train_df = train_df.fillna(0)
print("Columnas con valores NaN despues de rellenar:")
print(train_df.columns[train_df.isna().any()].tolist())

Columnas con valores NaN despues de rellenar:
[]


In [9]:
train_df.info() 

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 307511 entries, 0 to 307510
Columns: 662 entries, SK_ID_CURR to CCB_credit_card_balance_records_count
dtypes: float64(605), int64(41), object(16)
memory usage: 1.5+ GB


In [11]:
# OHE de columnas categóricas
# Identificar columnas categóricas
cat_cols = train_df.select_dtypes(include=["object"]).columns.tolist()
print("Columnas categóricas:", cat_cols)

# Aplicar One Hot Encoding
train_df = pd.get_dummies(train_df, columns=cat_cols, dummy_na=True)

train_df.shape

Columnas categóricas: ['NAME_CONTRACT_TYPE', 'CODE_GENDER', 'FLAG_OWN_CAR', 'FLAG_OWN_REALTY', 'NAME_TYPE_SUITE', 'NAME_INCOME_TYPE', 'NAME_EDUCATION_TYPE', 'NAME_FAMILY_STATUS', 'NAME_HOUSING_TYPE', 'OCCUPATION_TYPE', 'WEEKDAY_APPR_PROCESS_START', 'ORGANIZATION_TYPE', 'FONDKAPREMONT_MODE', 'HOUSETYPE_MODE', 'WALLSMATERIAL_MODE', 'EMERGENCYSTATE_MODE']


(307511, 808)

In [12]:
df_info_summary(train_df)   

                                                    Non-Null Count  Null Count  % Null    Dtype
SK_ID_CURR                                                  307511           0     0.0    int64
TARGET                                                      307511           0     0.0    int64
CNT_CHILDREN                                                307511           0     0.0    int64
AMT_INCOME_TOTAL                                            307511           0     0.0  float64
AMT_CREDIT                                                  307511           0     0.0  float64
AMT_ANNUITY                                                 307511           0     0.0  float64
AMT_GOODS_PRICE                                             307511           0     0.0  float64
REGION_POPULATION_RELATIVE                                  307511           0     0.0  float64
DAYS_BIRTH                                                  307511           0     0.0    int64
DAYS_EMPLOYED                           

In [18]:
# Dado que el preprocesamiento puede tardar, guardamos el objeto en memoria para futuro uso
# train_df.to_pickle(os.path.join(PATH, 'prework-tabular-object.pkl'))

# train_df.to_csv('merged_train.csv', index=False)

# Guardar df final en formato parquet
train_df.to_parquet(os.path.join(PATH, "train_df.parquet"))
